In [1]:
# Struktura souboru PUP_CUR_PVP_FVU_P_556598819493649678.geojson
import geopandas as gpd

# 1) Načteme GeoJSON
file_path = "PUP_CUR_PVP_FVU_P_556598819493649678.geojson"
gdf = gpd.read_file(file_path)

# 2) Seznam sloupců
columns = gdf.columns.tolist()
print("Sloupce v souboru:")
for col in columns:
    print(" •", col)

# 3) Ukázka prvních 5 řádků (atributy, bez geometrie)
print("\nPrvních 5 řádků (bez geometrií):")
print(gdf.drop(columns="geometry").head(50))


Sloupce v souboru:
 • OBJECTID
 • KODFP1_A
 • ZAS1
 • CSO1
 • KODFP2_A
 • ZAS2
 • CSO2
 • POPIS
 • POPIS_Z
 • ZDROJ
 • ZAS1_S
 • geometry

Prvních 5 řádků (bez geometrií):
    OBJECTID KODFP1_A     ZAS1     CSO1 KODFP2_A     ZAS2     CSO2 POPIS  \
0     159882      DGP  NO_DATA  NO_DATA  NO_DATA  NO_DATA  NO_DATA   DGP   
1     159883      DGP  NO_DATA  NO_DATA  NO_DATA  NO_DATA  NO_DATA   DGP   
2     159884      DGP  NO_DATA  NO_DATA  NO_DATA  NO_DATA  NO_DATA   DGP   
3     159885      DGP  NO_DATA  NO_DATA  NO_DATA  NO_DATA  NO_DATA   DGP   
4     159886      DGP  NO_DATA  NO_DATA  NO_DATA  NO_DATA  NO_DATA   DGP   
5     159887      DGP  NO_DATA  NO_DATA  NO_DATA  NO_DATA  NO_DATA   DGP   
6     159888      DGP  NO_DATA  NO_DATA  NO_DATA  NO_DATA  NO_DATA   DGP   
7     159889      DGP  NO_DATA  NO_DATA  NO_DATA  NO_DATA  NO_DATA   DGP   
8     159890      DGP  NO_DATA  NO_DATA  NO_DATA  NO_DATA  NO_DATA   DGP   
9     159891      DGP  NO_DATA  NO_DATA  NO_DATA  NO_DATA  NO_DATA  

In [1]:
# **** 1 ****  SQL query to extract Valuo and KN parcel data with specific conditions

query_pozemky_zzzZZZ = """
WITH ValidValuo AS (
    SELECT *
    FROM dbo.Valuo_data
    WHERE 
	    nemovitost = 'parcela'
	    and cislo_vkladu IN (
        SELECT cislo_vkladu
        FROM dbo.Valuo_data
        GROUP BY cislo_vkladu
/*      
	    HAVING 
            COUNT(*) = COUNT(CASE WHEN nemovitost = 'parcela' THEN 1 END)
            AND COUNT(CASE WHEN GPS_API_info = 'ERR' THEN 1 END) = 0

*/
    )



),
ParcelCounts AS (
    SELECT 
        V.cislo_vkladu,
        COUNT(DISTINCT V.adresa) AS ParcelCount
    FROM dbo.Valuo_data AS V
    LEFT JOIN dbo.KN_parcel_data AS K
        ON K.id_valuo = V.id
    GROUP BY V.cislo_vkladu
),
ValuoWithJC AS (
    SELECT 
        V.id                    AS id_db,
        V.cislo_vkladu,
        V.rok,
        V.mesic,
        V.datum_podani,
        V.listina,
        PC.ParcelCount          AS [#PARCEL],
        SUM(V.plocha) OVER (PARTITION BY V.cislo_vkladu) 
                                AS SUM_PARCEL_RIZENI,
        V.okres,
        V.kat_uzemi              AS KU_Valuo,
        K.zoning_title           AS KU_KN,
        K.upper_zoning_id        AS kod_ku,
        K.administrativeUnit_title 
                                 AS lokalita,
        V.nemovitost,
        V.typ,
        K.parcel_number,
        V.plocha,
        V.cenovy_udaj,
        CAST(
            ROUND(
                V.cenovy_udaj 
                / NULLIF(SUM(V.plocha) OVER (PARTITION BY V.cislo_vkladu), 0),
                0
            ) 
            AS DECIMAL(38,0)
        )                        AS JC_avg_V,
        K.gml_id,
        K.areaValue_m2,
        K.beginLifespanVersion,
        K.endLifespanVersion,
        K.geometry,
        K.inspire_localId,
        K.inspire_namespace,
        K.label,
        K.nationalCadastralReference,
        K.refPoint_x,
        K.refPoint_y,
        K.refPoint_lon,
        K.refPoint_lat,
        K.validFrom,
        K.administrativeUnit_href,
        K.administrativeUnit_title AS adminUnitTitle,
        K.zoning_href,
        K.zoning_title            AS zoningTitleUP,  -- tady se neplete se zoning_title z Valuo KN
        K.id_valuo
    FROM ValidValuo AS V
    LEFT JOIN dbo.KN_parcel_data AS K
        ON K.id_valuo = V.id
    LEFT JOIN ParcelCounts AS PC
        ON PC.cislo_vkladu = V.cislo_vkladu
)
SELECT
    ROW_NUMBER() OVER (ORDER BY datum_podani DESC) AS seq_id,
    *
FROM ValuoWithJC
WHERE 
    parcel_number is not NULL
	AND geometry is not NULL
    --AND JC_avg_V > 999
    --AND KU_Valuo IN ('Písnice', 'Kunratice', 'Libuš')


ORDER BY seq_id;
"""

query_pozemky = """
select * from [dbo].[KN_parcel_data] kn
left join [dbo].[Valuo_data] v on kn.id_valuo = v.id
where 1=1
      --and id_UP_FVU_data is null
	  and v.okres = 'Hlavní město Praha'
	  --and parcel_number is null
	  --and geometry is null

"""

In [4]:
# **** 2 **** Porovnava geometrie pozemku s polygony uzemniho planu a prirazuje pozemky k UP polygonum

import logging
import urllib.parse
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
from shapely import wkt
from sqlalchemy import create_engine, text

# ====================================================================
# 1) Logging
# ====================================================================
logger = logging.getLogger("ParcelUP_batch")
logger.setLevel(logging.DEBUG)
fmt = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
ch.setFormatter(fmt)
logger.addHandler(ch)

# ====================================================================
# 2) Předpokládáme, že v předchozím buňce máte query_pozemky definováno.
#    Pokud ne, musíte ho sem vložit ručně. 
# ====================================================================
try:
    query_pozemky
except NameError:
    raise NameError("Proměnná `query_pozemky` není definovaná. Definujte ji před spuštěním tohoto bloku.")

# Vytvoříme SQLAlchemy engine (pokud už neexistuje)
params = urllib.parse.quote_plus(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=localhost;"
    "Database=VALUO;"
    "Trusted_Connection=yes;"
)
connection_url = f"mssql+pyodbc:///?odbc_connect={params}"
engine = create_engine(connection_url)

# ====================================================================
# 3) Pomocné funkce pro převod
# ====================================================================

def parse_geometry(geom_str: str):
    """
    Vstup: textový řetězec 'x1 y1 x2 y2 ...', převod na shapely.geometry.Polygon.
    Pokud geom_str je None, vrátí None.
    """
    if geom_str is None:
        return None
    try:
        coords = list(map(float, geom_str.split()))
        return Polygon([(coords[i], coords[i+1]) for i in range(0, len(coords), 2)])
    except Exception as e:
        logger.error(f"Chyba při převodu geometry VALUO→POLYGON: '{geom_str[:50]}…': {e}")
        return None

def parse_wkt(wkt_str: str):
    """
    Vstup: WKT text 'POLYGON ((x1 y1, x2 y2, ...))'. 
    Používáme shapely.wkt.loads.
    """
    if wkt_str is None:
        return None
    try:
        return wkt.loads(wkt_str)
    except Exception as e:
        logger.error(f"Chyba při převodu WKT (UP_FVU_data): '{wkt_str[:50]}…': {e}")
        return None

# ====================================================================
# 4) Načtení a příprava parcel z query_pozemky → gdf_parcely (CRS 4326)
# ====================================================================
def load_and_prepare_parcels(sql_query: str, engine) -> gpd.GeoDataFrame:
    """
    1) Načte DataFrame df_p z dotazu `sql_query`.
    2) Parsuje sloupec 'geometry' (text) přes parse_geometry().
    3) Vytvoří GeoDataFrame (původní CRS EPSG:5514 → převede do EPSG:4326).
    """
    logger.info("Načítám data o parcelách (VALUO+KN) z databáze …")
    df_p = pd.read_sql(sql_query, engine)
    logger.info(f"Načteno {len(df_p)} řádků z Valuo+KN dotazu.")

    # Převod sloupce 'geometry' z textu na shapely Polygon
    df_p['geometry'] = df_p['geometry'].apply(parse_geometry)

    # Vytvoření GeoDataFrame. Předpokládáme, že původní CRS je EPSG:5514.
    gdf_p = gpd.GeoDataFrame(df_p, geometry='geometry', crs="EPSG:5514")
    # Převod na EPSG:4326
    gdf_p = gdf_p.to_crs(epsg=4326)
    logger.info(f"Parcely: nyní CRS {gdf_p.crs}, geometrií: {len(gdf_p)}")
    return gdf_p

# ====================================================================
# 5) Načtení polygonálních záznamů z tabulky UP_FVU_data
# ====================================================================
def load_up_polygons_from_db(engine) -> gpd.GeoDataFrame:
    """
    Načte z tabulky dbo.UP_FVU_data sloupce [id, KODFP1_A, geometry],
    kde geometry je WKT (typ TEXT). 
    Parsuje WKT na shapely Polygon, vytvoří GeoDataFrame s CRS EPSG:4326.
    """
    sql = """
    SELECT 
        id, 
        KODFP1_A, 
        geometry 
    FROM dbo.UP_FVU_data
    WHERE geometry IS NOT NULL
    """
    logger.info("Načítám všechny UP_FVU_data (id, KODFP1_A, geometry) z DB …")
    df_up = pd.read_sql(sql, engine)
    logger.info(f"Načteno {len(df_up)} řádků z UP_FVU_data.")

    # Parsujeme WKT text na shapely Polygon a uložíme to do sloupce 'up_geometry'
    df_up['up_geometry'] = df_up['geometry'].apply(parse_wkt)

    # Vytvoříme GeoDataFrame tak, že jako aktivní sloupec geometrie zvolíme 'up_geometry'
    gdf_up = gpd.GeoDataFrame(df_up, geometry='up_geometry', crs="EPSG:4326")

    # Odstraníme původní textový sloupec 'geometry' a přejmenujeme 'up_geometry' → 'geometry'
    gdf_up = (
        gdf_up
        .drop(columns=['geometry'])
        .rename(columns={'up_geometry': 'geometry'})
    )

    # Teď musíme přejmenovaný sloupec 'geometry' znovu nastavit jako aktivní geometrii
    gdf_up = gdf_up.set_geometry('geometry')

    logger.info(
        f"UP polygony: CRS je {gdf_up.crs}, počet {len(gdf_up)} polygonů, "
        f"sloupce: {gdf_up.columns.tolist()}"
    )
    return gdf_up

# ====================================================================
# 6) Prostorové přiřazení s výběrem polygonu s největší plochou průniku
# ====================================================================
def assign_up_with_max_intersection(
    gdf_parcely: gpd.GeoDataFrame,
    gdf_up: gpd.GeoDataFrame,
    engine
):
    """
    Pro každou parcelu v gdf_parcely najde ten polygon UP, ve kterém je 
    plošně největší část parcely (tj. maximalizuje plochu průniku).
    Výsledkem je hromadné UPDATE do KN_parcel_data.id_UP_FVU_data 
    s právě tímto up_id.
    """

    # ------------------------------------------------------------
    # 1) Ověření aktivního sloupce 'geometry' u obou GeoDataFrame
    # ------------------------------------------------------------
    if gdf_parcely.geometry.name != 'geometry':
        gdf_parcely = gdf_parcely.set_geometry('geometry')
    if gdf_up.geometry.name != 'geometry':
        gdf_up = gdf_up.set_geometry('geometry')

    # ------------------------------------------------------------
    # 2) Kontrola, že obě GeoDataFrame mají stejný CRS (EPSG:4326)
    # ------------------------------------------------------------
    if (gdf_parcely.crs is None 
        or gdf_up.crs is None 
        or (gdf_parcely.crs.to_epsg() != gdf_up.crs.to_epsg())):
        raise ValueError(f"CRS nesouhlasí: parcely={gdf_parcely.crs}, UP={gdf_up.crs}")

    logger.info("Provádím prostorový join (intersects) mezi parcelami a UP polygony …")
    joined = gpd.sjoin(
        gdf_parcely,
        gdf_up[['id', 'KODFP1_A', 'geometry']],
        how="inner",         # Vezmeme pouze páry, které se skutečně protínají
        predicate="intersects"
    )
    logger.info(f"Navázáno {len(joined)} parcel × UP polygonů (včetně duplicity parcel).")

    # ------------------------------------------------------------
    # 3) Detekce názvu sloupce s ID z UP (může být 'id' nebo 'id_right')
    # ------------------------------------------------------------
    if 'id' in joined.columns:
        up_id_col = 'id'
    elif 'id_right' in joined.columns:
        up_id_col = 'id_right'
    else:
        raise KeyError(
            f"Ve výsledném joined neexistuje ani 'id' ani 'id_right'. "
            f"Sloupce: {joined.columns.tolist()}"
        )

    # ------------------------------------------------------------
    # 4) Převod do metricky korektního CRS (např. EPSG:5514) pro výpočet plochy
    # ------------------------------------------------------------
    gdf_parcely_m = gdf_parcely.to_crs(epsg=5514)
    gdf_up_m      = gdf_up.to_crs(epsg=5514)

    # Uchování původních indexů pro dohledání geometrií v metricky korektním CRS
    joined = joined.reset_index().rename(columns={'index': 'orig_index_parcela'})
    joined = joined.rename(columns={'index_right': 'orig_index_up'})  # Upravte, pokud se jmenuje jinak

    # ------------------------------------------------------------
    # 5) Dohledání přesné geometrie parcely a UP polygonu v CRS EPSG:5514
    # ------------------------------------------------------------
    joined['geom_parcela_m'] = joined['orig_index_parcela'].apply(
        lambda idx: gdf_parcely_m.at[idx, 'geometry']
    )
    joined['geom_up_m'] = joined['orig_index_up'].apply(
        lambda idx: gdf_up_m.at[idx, 'geometry']
    )

    # ------------------------------------------------------------
    # 6) Výpočet průnikové geometrie a plochy (v m²)
    # ------------------------------------------------------------
    logger.info("Počítám geometrii průniku a plochu průniku (v m²) pro každý pár (parcela, UP).")
    joined['intersection_geom'] = joined.apply(
        lambda row: row['geom_parcela_m'].intersection(row['geom_up_m']),
        axis=1
    )
    joined['intersection_area'] = joined['intersection_geom'].area

    # ------------------------------------------------------------
    # 7) Výběr jednoho řádku s maximální průnikovou plochou pro každou parcelu
    # ------------------------------------------------------------
    idx_max = (
        joined
        .loc[joined['intersection_area'] > 0]
        .groupby('parcel_number')['intersection_area']
        .idxmax()
    )

    joined_best = joined.loc[idx_max, ['parcel_number', up_id_col]].copy()
    joined_best = joined_best.rename(columns={up_id_col: 'up_id'})

    logger.info(f"Po výběru maxima zůstalo {len(joined_best)} parcel (každá s jednou největší plochou).")

    # ------------------------------------------------------------
    # 8) Hromadný UPDATE do databáze KN_parcel_data.id_UP_FVU_data
    # ------------------------------------------------------------
    conn = engine.connect()
    trans = conn.begin()
    try:
        for _, row in joined_best.iterrows():
            parc_num = row['parcel_number']
            up_id    = int(row['up_id'])
            update_sql = text("""
                UPDATE dbo.KN_parcel_data
                SET id_UP_FVU_data = :up_id
                WHERE parcel_number = :parc_num
            """)
            conn.execute(update_sql, {"up_id": up_id, "parc_num": parc_num})
        trans.commit()
        logger.info("Hromadné UPDATE do KN_parcel_data (id_UP_FVU_data) proběhlo v pořádku.")
    except Exception as e:
        trans.rollback()
        logger.error(f"Při UPDATE do KN_parcel_data došlo k chybě: {e}")
    finally:
        conn.close()

# ====================================================================
# 7) Spustíme celý tok
# ====================================================================
def main():
    # 7.1) Načtěte GeoDataFrame parcel z předdefinovaného dotazu
    gdf_parcely = load_and_prepare_parcels(query_pozemky, engine)

    # 7.2) Načtěte UP polygony z databáze
    gdf_up_polygony = load_up_polygons_from_db(engine)

    # 7.3) Prostorové přiřazení s výběrem maximální plochy průniku
    assign_up_with_max_intersection(
        gdf_parcely=gdf_parcely,
        gdf_up=gdf_up_polygony,
        engine=engine
    )

if __name__ == "__main__":
    main()


2025-06-02 18:48:36,876 - INFO - Načítám data o parcelách (VALUO+KN) z databáze …
2025-06-02 18:48:36,876 - INFO - Načítám data o parcelách (VALUO+KN) z databáze …
2025-06-02 18:48:36,876 - INFO - Načítám data o parcelách (VALUO+KN) z databáze …


2025-06-02 18:48:40,630 - INFO - Načteno 29107 řádků z Valuo+KN dotazu.
2025-06-02 18:48:40,630 - INFO - Načteno 29107 řádků z Valuo+KN dotazu.
2025-06-02 18:48:40,630 - INFO - Načteno 29107 řádků z Valuo+KN dotazu.
2025-06-02 18:48:41,876 - INFO - Parcely: nyní CRS EPSG:4326, geometrií: 29107
2025-06-02 18:48:41,876 - INFO - Parcely: nyní CRS EPSG:4326, geometrií: 29107
2025-06-02 18:48:41,876 - INFO - Parcely: nyní CRS EPSG:4326, geometrií: 29107
2025-06-02 18:48:41,899 - INFO - Načítám všechny UP_FVU_data (id, KODFP1_A, geometry) z DB …
2025-06-02 18:48:41,899 - INFO - Načítám všechny UP_FVU_data (id, KODFP1_A, geometry) z DB …
2025-06-02 18:48:41,899 - INFO - Načítám všechny UP_FVU_data (id, KODFP1_A, geometry) z DB …
2025-06-02 18:48:42,085 - INFO - Načteno 15542 řádků z UP_FVU_data.
2025-06-02 18:48:42,085 - INFO - Načteno 15542 řádků z UP_FVU_data.
2025-06-02 18:48:42,085 - INFO - Načteno 15542 řádků z UP_FVU_data.
2025-06-02 18:48:42,449 - INFO - UP polygony: CRS je EPSG:4326, 

In [ ]:
# Vlozeni CSV souboru s udaji o funkcnim vyuziti ploch dle UP Prahy do DB

import pandas as pd
import pyodbc

# ----------------------------------------
# Nastavení parametrů připojení k SQL Serveru
# ----------------------------------------
SERVER   = 'localhost'      # např. 'localhost\\SQLEXPRESS' nebo IP adresa
DATABASE = 'VALUO'          # jméno databáze, kde je UP_FVU_data

conn_str = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    f'SERVER={SERVER};'
    f'DATABASE={DATABASE};'
    'Trusted_Connection=yes;'
)

# ----------------------------------------
# 1) Funkce pro vložení jednoho "batch" řádků
# ----------------------------------------
def insert_batch(cursor, rows):
    """
    rows: seznam n-tic, každá n-tice odpovídá 12 hodnotám ve sloupcích:
    (OBJECTID, KODFP1_A, ZAS1, CSO1, KODFP2_A, ZAS2, CSO2,
     POPIS, POPIS_Z, ZDROJ, ZAS1_S, geometry)
    """
    insert_sql = """
        INSERT INTO dbo.UP_FVU_data
        (
            OBJECTID,
            KODFP1_A,
            ZAS1,
            CSO1,
            KODFP2_A,
            ZAS2,
            CSO2,
            POPIS,
            POPIS_Z,
            ZDROJ,
            ZAS1_S,
            geometry
        )
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
    """
    cursor.fast_executemany = True
    cursor.executemany(insert_sql, rows)

# ----------------------------------------
# 2) Načtení a vložení po částech (chunks)
# ----------------------------------------
csv_path = r'C:\Users\ijttr\OneDrive\Dokumenty\PROG\PYTHON\DATA_ANALYSIS\VALUO\gdf.csv'

# Zmenšete chunk_size, pokud stále dochází k MemoryError (např. 1000, 500 nebo i 100)
chunk_size = 500  

expected_cols = [
    'OBJECTID', 'KODFP1_A', 'ZAS1', 'CSO1', 'KODFP2_A', 'ZAS2', 'CSO2',
    'POPIS', 'POPIS_Z', 'ZDROJ', 'ZAS1_S', 'geometry'
]

conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

total_inserted = 0

for chunk_df in pd.read_csv(csv_path, encoding='utf-8', chunksize=chunk_size):
    # Ověř, že chunk obsahuje požadované sloupce
    missing = [col for col in expected_cols if col not in chunk_df.columns]
    if missing:
        raise ValueError(f"Chybí sloupce v CSV: {missing}")
    
    # Převod NaN → None
    records = chunk_df[expected_cols].astype(object).where(pd.notnull(chunk_df), None)
    rows = [tuple(row) for row in records.to_numpy()]
    
    # Pokus o vložení batchu; pokud MemoryError, rozdělíme na menší části
    try:
        insert_batch(cursor, rows)
        conn.commit()
        total_inserted += len(rows)
        print(f"Vloženo řádků v tomto batchi: {len(rows)}; Celkem dosud vloženo: {total_inserted}")
    except MemoryError:
        # Pokud stále nedostatek paměti, vkládej po řádcích
        inserted = 0
        for row in rows:
            cursor.execute(
                """
                INSERT INTO dbo.UP_FVU_data
                (
                    OBJECTID,
                    KODFP1_A,
                    ZAS1,
                    CSO1,
                    KODFP2_A,
                    ZAS2,
                    CSO2,
                    POPIS,
                    POPIS_Z,
                    ZDROJ,
                    ZAS1_S,
                    geometry
                )
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
                """,
                row
            )
            inserted += 1
            if inserted % 100 == 0:
                conn.commit()
        conn.commit()
        total_inserted += inserted
        print(f"Vloženo po řádcích (batch) {inserted} řádků; Celkem dosud vloženo: {total_inserted}")
    
    # Uvolnění paměti
    del chunk_df, records, rows

cursor.close()
conn.close()

print(f"\nImport dokončen. Celkový počet vložených řádků: {total_inserted}")


Vloženo po řádcích (batch) 500 řádků; Celkem dosud vloženo: 500
Vloženo po řádcích (batch) 500 řádků; Celkem dosud vloženo: 1000
Vloženo po řádcích (batch) 500 řádků; Celkem dosud vloženo: 1500
Vloženo po řádcích (batch) 500 řádků; Celkem dosud vloženo: 2000
Vloženo po řádcích (batch) 500 řádků; Celkem dosud vloženo: 2500
Vloženo po řádcích (batch) 500 řádků; Celkem dosud vloženo: 3000
Vloženo po řádcích (batch) 500 řádků; Celkem dosud vloženo: 3500
Vloženo po řádcích (batch) 500 řádků; Celkem dosud vloženo: 4000
Vloženo po řádcích (batch) 500 řádků; Celkem dosud vloženo: 4500
Vloženo po řádcích (batch) 500 řádků; Celkem dosud vloženo: 5000
Vloženo po řádcích (batch) 500 řádků; Celkem dosud vloženo: 5500
Vloženo po řádcích (batch) 500 řádků; Celkem dosud vloženo: 6000
Vloženo po řádcích (batch) 500 řádků; Celkem dosud vloženo: 6500
Vloženo po řádcích (batch) 500 řádků; Celkem dosud vloženo: 7000
Vloženo po řádcích (batch) 500 řádků; Celkem dosud vloženo: 7500
Vloženo po řádcích (batch)